In [2]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import OneHotEncoder

from helpers import custom_train_test_split
from mlp import MLP
from layers.average_pooling_layer import AvgPool2D
from layers.flatten import Flatten
from layers.conv2d import Conv2D
from layers.dense import DenseLayer
from activations.softmax import Softmax
from losses.categorical_cross_entropy import CategoricalCrossEntropy


# np.random.seed(33)

mnist = fetch_openml("mnist_784", version=1, parser="auto")

x, y = mnist.data, mnist.target

x = x / 255.0

x = x.values.reshape(-1, 1, 28, 28)
x = np.pad(x, ((0, 0), (0, 0), (2, 2), (2, 2)), "constant", constant_values=0)

y = y.values.astype(np.int8)
encoder = OneHotEncoder(sparse_output=False, categories="auto")
y_encoded = encoder.fit_transform(y.reshape(-1, 1))

x_train, x_test, y_train, y_test = custom_train_test_split(x, y_encoded, test_size=0.2, random_state=42)

layers = [
        Conv2D(in_channels=1, out_channels=6, kernel_size=5),
        AvgPool2D(pool_size=2, stride=2),
        Conv2D(in_channels=6, out_channels=16, kernel_size=5),
        AvgPool2D(pool_size=2, stride=2),
        Conv2D(in_channels=16, out_channels=120, kernel_size=5),
        Flatten(),
        DenseLayer(input_size=None, output_size=84),
        DenseLayer(input_size=84, output_size=10),
        Softmax()
]
loss_fn = CategoricalCrossEntropy()
model = MLP(layers, loss_fn)

batch_size = 100

model.train(x_train, y_train, epochs=10, batch_size=batch_size, learning_rate=0.005)

    # x_test = x_test.reshape(-1, 1, 1, 28, 28)
print("True:   ", np.argmax(y_test[25:55], axis=-1))
print("Predict:", np.argmax(model.forward(x_test[25:55]), axis=-1))


Training Progress: 100%|██████████| 10/10 [08:12<00:00, 49.25s/it, Epoch=10, Train Mean Loss=2.1696]

True:    [3 6 1 6 9 7 6 0 8 5 5 2 9 1 6 4 9 7 7 4 5 2 7 3 9 2 0 6 8 5]
Predict: [3 6 1 6 9 7 6 0 8 5 5 2 9 1 6 4 9 9 7 4 5 8 7 3 9 2 0 6 8 5]


In [ ]:
print(MLP.calculate_precision_recall(y_test, model.forward(x_test)))
print(f"F1 Score after train: {model.calculate_f1_score(y_train, model.forward(x_train))}")
print(f"F1 Score after test: {model.calculate_f1_score(y_test, model.forward(x_test))}")
MLP.draw_roc_curve(y_test, model.forward(x_test))


([0.9422514619883041, 0.9054290718038529, 0.8850405305821666, 0.8568435754189944, 0.8816479400749063, 0.8646179401993356, 0.9026915113871635, 0.9022950819672131, 0.9013377926421404, 0.8599577762139339], [0.9597915115413254, 0.969375, 0.8702898550724638, 0.8562456385205862, 0.9088803088803089, 0.8177533385703064, 0.9369627507163324, 0.9155023286759814, 0.7943994104642594, 0.8605633802816901])
